# Loading importing

In [1]:
!nvidia-smi

import os
import argparse
import datetime
import time
import cv2
import random
import torch
import numpy as np
import glob
import matplotlib.pyplot as plt


os.environ['CUDA_VISIBLE_DEVICES'] = '0'

Fri Sep 16 19:29:12 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.141.03   Driver Version: 470.141.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    Off  | 00000000:18:00.0 Off |                  Off |
| 59%   87C    P2   277W / 300W |  10311MiB / 48685MiB |    100%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA RTX A6000    Off  | 00000000:3B:00.0 Off |                  Off |
|100%   

In [2]:
# fix random seeds for reproducibility
random_seed = 42
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) 
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

P: 75 (22.73%) N: 255 (77.27%) Total: 330
P: 14 (20.00%) N: 56  (80.00%) Total: 70
P: 21 (21.00%) N: 79  (79.00%) Total: 100

# Dataset

In [3]:
pwd

'/workspace/sunggu/7.KOHI/Multi_task_learning_tutorials'

In [4]:
DATA_DIR = './SSIM_seg/'

x_train_list = glob.glob(os.path.join(DATA_DIR, 'train/*'))
y_train_list = glob.glob(os.path.join(DATA_DIR, 'trainannot/*'))

x_valid_list = glob.glob(os.path.join(DATA_DIR, 'val/*'))
y_valid_list = glob.glob(os.path.join(DATA_DIR, 'valannot/*'))

x_test_list  = glob.glob(os.path.join(DATA_DIR, 'test/*'))
y_test_list  = glob.glob(os.path.join(DATA_DIR, 'testannot/*'))

In [5]:
x_train_list[0]

'./SSIM_seg/train/1.2.276.0.7230010.3.1.4.8323329.300.1517875162.258081.png'

In [6]:
x_valid_list[0]

'./SSIM_seg/val/1.2.276.0.7230010.3.1.4.8323329.1173.1517875166.626582.png'

In [7]:
from torch.utils.data import Dataset as BaseDataset
from torch.utils.data import DataLoader


# Dataset Class
class Dataset(BaseDataset):    
    def __init__(self, images_list, labels_list, transform):
        self.images_list  = images_list
        self.labels_list  = labels_list
        self.transform    = transform
    
    def __getitem__(self, i):
        # read data
        image = cv2.imread(self.images_list[i], cv2.IMREAD_GRAYSCALE)
        mask  = cv2.imread(self.labels_list[i], cv2.IMREAD_GRAYSCALE)
        path  = self.images_list[i]
        
        mask  = np.expand_dims(mask, axis=0)
            
        # apply transform
        sample = self.transform(image=image, mask=mask)
        image, mask = sample['image'], sample['mask']        
        
        return image, mask, path
        
    def __len__(self):
        return len(self.images_list)

In [8]:
from albumentations.pytorch.transforms import ToTensorV2
import albumentations as albu


def minmax_normalize(image, **kwargs):
    if len(np.unique(image)) != 1:  # Sometimes it cause the nan inputs...
        image = image.astype('float32')
        image -= image.min()
        image /= image.max() 
    return image


train_transform = albu.Compose([
    albu.HorizontalFlip(p=0.5),
    albu.ShiftScaleRotate(scale_limit=0.10, shift_limit=0.10, rotate_limit=15, p=0.5),
    albu.GaussNoise(p=0.2),
    albu.OneOf(
        [
            albu.CLAHE(p=1),
            albu.RandomBrightnessContrast(p=1),
            albu.RandomGamma(p=1),
        ],
        p=0.3,
    ),
    albu.OneOf(
        [
            albu.Blur(blur_limit=3, p=1),
            albu.MotionBlur(blur_limit=3, p=1),
        ],
        p=0.3,
    ),
    albu.Lambda(image=minmax_normalize, always_apply=True),
    ToTensorV2(),    
])

valid_transform = albu.Compose([        
    albu.Lambda(image=minmax_normalize, always_apply=True),
    ToTensorV2(),
])


In [9]:
dataset_train = Dataset(images_list=x_train_list, labels_list=y_train_list, transform=train_transform)
dataset_valid = Dataset(images_list=x_valid_list, labels_list=y_valid_list, transform=valid_transform)

data_loader_train = torch.utils.data.DataLoader(dataset_train, batch_size=10, num_workers=4, shuffle=True, pin_memory=True, drop_last=True)
data_loader_valid = torch.utils.data.DataLoader(dataset_valid, batch_size=1,  num_workers=4, shuffle=True, pin_memory=True, drop_last=False)

In [ ]:
# # same image with different random transforms

# batch = next(iter(train_loader))
# x = batch['x'][0]
# y_seg = batch['y_seg'][0]
# y_cls = batch['y_cls'][0]

# print(x.shape,y_seg.shape,y_cls.shape)
# print(torch.unique(y_seg),y_cls)
# visualize(image=x, mask=y_seg)

# Model

In [10]:
from arch.smart_net import *
from losses import MTL_Loss

# Model
model        = MTL_1_Net(encoder_name='resnet18').to('cuda')     

# Loss
criterion    = MTL_Loss(name='MTL_1')

# Optimizer & LR Schedule   
optimizer    = torch.optim.AdamW(params=model.parameters(), lr=1e-4, betas=(0.9, 0.999), eps=1e-08, weight_decay=5e-4, amsgrad=False)

n_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
print('Number of Learnable Params:', n_parameters)   

Number of Learnable Params: 14322711


# Loop

In [11]:
import json
import math
import torch
import utils
from torch import nn
import torch.nn.functional as F

from metrics import *
from losses import soft_dice_score
from sklearn.metrics import roc_auc_score

print_freq = 10
output_dir = './checkpoints/mtl_1/'
device     = 'cuda'

# Whole LOOP
for epoch in range(0, 200):
    
    ################################################################################################
    # Training 
    ################################################################################################
    
    model.train(True)
    metric_logger = utils.MetricLogger(delimiter="  ", n=10)
    metric_logger.add_meter('lr', utils.SmoothedValue(window_size=1, fmt='{value:.6f}'))
    header = 'Epoch: [{}]'.format(epoch)
    
    for batch_data in metric_logger.log_every(data_loader_train, print_freq, header):
        
        input   = batch_data[0].to(device).float()
        seg_gt  = batch_data[1].to(device).float()
        cls_gt  = seg_gt.flatten(1).bool().any(dim=1).float().unsqueeze(1)
        
        cls_pred, seg_pred = model(input)

        loss, loss_detail = criterion(cls_pred=cls_pred, seg_pred=seg_pred, cls_gt=cls_gt, seg_gt=seg_gt)
        
        loss_value = loss.item()

        if not math.isfinite(loss_value):
            print("Loss is {}, stopping training".format(loss_value))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        metric_logger.update(loss=loss_value)
        if loss_detail is not None:
            metric_logger.update(**loss_detail)
        metric_logger.update(lr=optimizer.param_groups[0]["lr"])
    
    train_stats = {k: round(meter.global_avg, 7) for k, meter in metric_logger.meters.items()}
    print("Averaged train_stats: ", train_stats)
    
    ################################################################################################
    # Validation
    ################################################################################################
    
    model.eval()
    metric_logger = utils.MetricLogger(delimiter="  ", n=1)
    header = 'Valid:'

    total_cls_pred  = torch.tensor([])
    total_cls_true  = torch.tensor([])
    
    for batch_data in metric_logger.log_every(data_loader_valid, print_freq, header):
        
        input   = batch_data[0].to(device).float()
        seg_gt  = batch_data[1].to(device).float()
        cls_gt  = seg_gt.flatten(1).bool().any(dim=1).float().unsqueeze(1)

        cls_pred, seg_pred = model(input)

        loss, loss_detail = criterion(cls_pred=cls_pred, seg_pred=seg_pred, cls_gt=cls_gt, seg_gt=seg_gt)
    
        loss_value = loss.item()

        if not math.isfinite(loss_value):
            print("Loss is {}, stopping training".format(loss_value))

        # LOSS
        metric_logger.update(loss=loss_value)        
        if loss_detail is not None:
            metric_logger.update(**loss_detail)

        # Post-processing
        cls_pred = torch.sigmoid(cls_pred)
        seg_pred = torch.sigmoid(seg_pred)

        total_cls_pred  = torch.cat([total_cls_pred, cls_pred.detach().cpu()])
        total_cls_true  = torch.cat([total_cls_true, cls_gt.detach().cpu()])

        # Metrics SEG
        if seg_gt.any():
            dice = soft_dice_score(output=seg_pred.round(), target=seg_gt, smooth=0.0)    # pred_seg must be round() !! 
            metric_logger.update(dice=dice.item())     

    # Metric CLS
    auc            = roc_auc_score(y_true=total_cls_true, y_score=total_cls_pred)
    tp, fp, fn, tn = get_stats(total_cls_pred.round().long(), total_cls_true.long(), mode="binary")        
    f1             = f1_score(tp, fp, fn, tn, reduction="macro")
    acc            = accuracy(tp, fp, fn, tn, reduction="macro")
    sen            = sensitivity(tp, fp, fn, tn, reduction="macro")
    spe            = specificity(tp, fp, fn, tn, reduction="macro")

    metric_logger.update(auc=auc, f1=f1, acc=acc, sen=sen, spe=spe)          
    
    valid_stats = {k: round(meter.global_avg, 7) for k, meter in metric_logger.meters.items()}
    print("Averaged valid_stats: ", valid_stats)
    
    ################################################################################################
    # Save & Log
    ################################################################################################
    
    checkpoint_paths = output_dir + '/epoch_' + str(epoch) + '_checkpoint.pth'
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'epoch': epoch,
    }, checkpoint_paths)

    log_stats = {**{f'train_{k}': v for k, v in train_stats.items()},
                **{f'valid_{k}': v for k, v in valid_stats.items()},
                'epoch': epoch}

    if output_dir:
        with open(output_dir + "/log.txt", "a") as f:
            f.write(json.dumps(log_stats) + "\n")

Epoch: [0]  [ 0/33]  eta: 0:00:29  lr: 0.000100  loss: 1.0010 (1.0010)  CLS_Loss: 0.7267 (0.7267)  SEG_Loss: 0.2744 (0.2744)  time: 0.8952  data: 0.5888  max mem: 2750
Epoch: [0]  [10/33]  eta: 0:00:06  lr: 0.000100  loss: 0.7641 (0.8349)  CLS_Loss: 0.6164 (0.6142)  SEG_Loss: 0.1442 (0.2207)  time: 0.2833  data: 0.0536  max mem: 2917
Epoch: [0]  [20/33]  eta: 0:00:03  lr: 0.000100  loss: 0.9044 (0.9341)  CLS_Loss: 0.5497 (0.5999)  SEG_Loss: 0.2480 (0.3342)  time: 0.2513  data: 0.0002  max mem: 2917
Epoch: [0]  [30/33]  eta: 0:00:00  lr: 0.000100  loss: 0.9044 (0.8909)  CLS_Loss: 0.5453 (0.5730)  SEG_Loss: 0.3503 (0.3179)  time: 0.2579  data: 0.0002  max mem: 2917
Epoch: [0]  [32/33]  eta: 0:00:00  lr: 0.000100  loss: 0.9044 (0.8999)  CLS_Loss: 0.4534 (0.5686)  SEG_Loss: 0.4179 (0.3313)  time: 0.2545  data: 0.0001  max mem: 2917
Epoch: [0] Total time: 0:00:08 (0.2681 s / it)
Averaged train_stats:  {'lr': 0.0001, 'loss': 0.8999225, 'CLS_Loss': 0.5685978, 'SEG_Loss': 0.3313247}
Valid:  [ 

Epoch: [3]  [30/33]  eta: 0:00:00  lr: 0.000100  loss: 0.6760 (0.7243)  CLS_Loss: 0.4935 (0.5505)  SEG_Loss: 0.2181 (0.1738)  time: 0.2701  data: 0.0001  max mem: 3309
Epoch: [3]  [32/33]  eta: 0:00:00  lr: 0.000100  loss: 0.6512 (0.6982)  CLS_Loss: 0.4989 (0.5469)  SEG_Loss: 0.0982 (0.1513)  time: 0.2920  data: 0.0001  max mem: 3309
Epoch: [3] Total time: 0:00:07 (0.2352 s / it)
Averaged train_stats:  {'lr': 0.0001, 'loss': 0.6981643, 'CLS_Loss': 0.5468816, 'SEG_Loss': 0.1512827}
Valid:  [ 0/70]  eta: 0:00:31  loss: 0.1905 (0.1905)  CLS_Loss: 0.1905 (0.1905)  SEG_Loss: 0.0000 (0.0000)  time: 0.4562  data: 0.4195  max mem: 3309
Valid:  [10/70]  eta: 0:00:04  loss: 0.1587 (0.4902)  CLS_Loss: 0.1587 (0.6208)  SEG_Loss: 0.0000 (-0.1306)  dice: 1.4587 (1.5510)  time: 0.0695  data: 0.0383  max mem: 3309
Valid:  [20/70]  eta: 0:00:02  loss: 0.1476 (0.4775)  CLS_Loss: 0.1476 (0.5726)  SEG_Loss: 0.0000 (-0.0950)  dice: 1.4587 (1.4596)  time: 0.0297  data: 0.0002  max mem: 3309
Valid:  [30/70] 

Valid:  [ 0/70]  eta: 0:00:30  loss: 0.2276 (0.2276)  CLS_Loss: 0.2276 (0.2276)  SEG_Loss: 0.0000 (0.0000)  time: 0.4290  data: 0.4055  max mem: 3309
Valid:  [10/70]  eta: 0:00:03  loss: 0.1208 (0.5122)  CLS_Loss: 0.1208 (0.5194)  SEG_Loss: 0.0000 (-0.0073)  dice: 0.5951 (1.1323)  time: 0.0561  data: 0.0371  max mem: 3309
Valid:  [20/70]  eta: 0:00:01  loss: 0.0822 (0.3241)  CLS_Loss: 0.0822 (0.3279)  SEG_Loss: 0.0000 (-0.0038)  dice: 0.5951 (1.1323)  time: 0.0183  data: 0.0002  max mem: 3309
Valid:  [30/70]  eta: 0:00:01  loss: 0.1138 (0.2964)  CLS_Loss: 0.1138 (0.3217)  SEG_Loss: 0.0000 (-0.0253)  dice: 1.6696 (1.3388)  time: 0.0179  data: 0.0001  max mem: 3309
Valid:  [40/70]  eta: 0:00:00  loss: 0.1418 (0.2623)  CLS_Loss: 0.1418 (0.2814)  SEG_Loss: 0.0000 (-0.0191)  dice: 1.6696 (1.3388)  time: 0.0181  data: 0.0001  max mem: 3309
Valid:  [50/70]  eta: 0:00:00  loss: 0.1418 (0.3119)  CLS_Loss: 0.1418 (0.3486)  SEG_Loss: 0.0000 (-0.0367)  dice: 1.2493 (1.3722)  time: 0.0183  data: 0.

KeyboardInterrupt: 

# Log check

In [ ]:
import glob
import numpy as np
import matplotlib.pyplot as plt

def read_log(path):
    log_list = []
    lines = open(path, 'r').read().splitlines() 
    for i in range(len(lines)):
        exec('log_list.append('+lines[i] + ')')
    return  log_list

In [ ]:
51, 36, 35

In [ ]:
log_list = read_log(path = '/workspace/sunggu/6.Kakao/checkpoints/[Baseline]Face_Net_DAC_fold0_upsample/log.txt')
print(log_list[0].keys())
result_dict = {}

for key in log_list[0].keys():
    exec( "result_dict['"+str(key)+"']" + " = [ log_list[i]['"+str(key)+"'] for i in range(len(log_list)) ]")

for key in result_dict.keys():
    plt.plot(result_dict[key])
    plt.title(key)
    print("###########################################################")
    print("Argsort = ", np.argsort(result_dict[key])[:5])
    print("Value   = ", [result_dict[key][i] for i in np.argsort(result_dict[key])[:5]])
    plt.show()
    
    if key == 'valid_loss':
        print("Valid_Loss = ", np.argsort(result_dict[key])[:3])

In [ ]:
# Log check

import glob
import numpy as np
import matplotlib.pyplot as plt

def read_log(path):
    log_list = []
    lines = open(path, 'r').read().splitlines() 
    for i in range(len(lines)):
        exec('log_list.append('+lines[i] + ')')
    return  log_list

51, 36, 35

log_list = read_log(path = '/workspace/sunggu/6.Kakao/checkpoints/[Baseline]Face_Net_DAC_fold0_upsample/log.txt')
print(log_list[0].keys())
result_dict = {}

for key in log_list[0].keys():
    exec( "result_dict['"+str(key)+"']" + " = [ log_list[i]['"+str(key)+"'] for i in range(len(log_list)) ]")

for key in result_dict.keys():
    plt.plot(result_dict[key])
    plt.title(key)
    print("###########################################################")
    print("Argsort = ", np.argsort(result_dict[key])[:5])
    print("Value   = ", [result_dict[key][i] for i in np.argsort(result_dict[key])[:5]])
    plt.show()
    
    if key == 'valid_loss':
        print("Valid_Loss = ", np.argsort(result_dict[key])[:3])

# TEST

In [ ]:
# Resume

print("Loading... Resume")
checkpoint = torch.load(args.resume, map_location='cpu')
model.load_state_dict(checkpoint['model_state_dict'])        
optimizer.load_state_dict(checkpoint['optimizer'])
lr_scheduler.load_state_dict(checkpoint['lr_scheduler'])        
args.start_epoch = checkpoint['epoch'] + 1  
try:
    log_path = os.path.dirname(args.resume)+'/log.txt'
    lines    = open(log_path,'r').readlines()
    val_loss_list = []
    for l in lines:
        exec('log_dict='+l.replace('NaN', '0'))
        val_loss_list.append(log_dict['valid_loss'])
    print("Epoch: ", np.argmin(val_loss_list), " Minimum Val Loss ==> ", np.min(val_loss_list))